In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import trange, notebook
from bs4 import BeautifulSoup
import requests, time
import random

In [2]:
user_id_list=open('./1119_data/steam_id.txt','r')
user_id = user_id_list.readlines()
user_id = list(map(lambda s: s.strip(), user_id))
user_id_list.close()

In [3]:
def del_app_detail(dic):    
    for i in dic:
        for j in range(len(dic[i])):
            try:
                if ('img_icon_url' in dic[i][j]):
                    del (dic[i][j])['img_icon_url']
            except:
                pass
            try:
                if ('img_logo_url' in dic[i][j]):
                    del (dic[i][j])['img_logo_url']
            except:
                pass
            try:
                if ('playtime_linux_forever' in dic[i][j]):
                    del (dic[i][j])['playtime_linux_forever']
            except:
                pass
            try:
                if ('playtime_windows_forever' in dic[i][j]):
                    del (dic[i][j])['playtime_windows_forever']
            except:
                pass
            try:
                if ('playtime_mac_forever' in dic[i][j]):
                    del (dic[i][j])['playtime_mac_forever']
            except:
                pass
            try:
                if ('has_community_visible_stats' in dic[i][j]):
                    del (dic[i][j])['has_community_visible_stats']
            except:
                pass
    return dic

In [4]:
def column_attr(init_df,name_lst):
    temp_set=set()
    for index in range(len(init_df)):
        for name in range(init_df.shape[1]):
            try:
                if type(init_df.loc[index][name])==type({}):
                    temp_name=init_df.loc[index][name]["name"]
                    temp_set.add(temp_name)
            except:
                continue
    for value in temp_set:
        name_lst.append(value)

In [5]:
def user_own_game(init_df,new_df,game_lst):
    for game_name in notebook.tqdm(game_lst):
        temp_lst=[]
        for index in range(len(init_df)):
            count=1
            found=False
            while(count<11):
                for name in range(10):
                    try:
                        if json.loads(init_df.loc[index][str(name)].replace("'","\""))["name"]==game_name:
                            temp_lst.append(json.loads(init_df.loc[index][str(name)].replace("'","\""))["playtime_forever"])
                            count=11
                            found=True
                        else:
                            count+=1
                    except:
                        count+=1
                        continue
            if found==True:
                continue
            else:
                temp_lst.append(None)
        new_df[str(game_name)]=temp_lst

In [ ]:
def user_own_game(init_df,new_df,game_lst):
    for game_name in notebook.tqdm(game_lst):
        for index in range(len(init_df)):
            temp_lst=[]
            count=1
            found=False
            while(count<init_df.shape[1]):
                for name in range(init_df.shape[1]):
                    try:
                        if init_df.loc[index][name]["name"]==game_name:
                            if init_df.loc[index][name]["playtime_forever"]>100:
                                temp_lst.append(init_df.loc[index][name]["playtime_forever"])
                                count=init_df.shape[1]
                                found=True
                            else:
                                count=init_df.shape[1]
                            count+=1
                    except:
                        count+=1
                        continue
            if found==True:
                continue
            else:
                temp_lst.append(None)
    new_df[str(game_name)]=temp_lst

In [30]:
df.loc[0][1]["playtime_forever"]

1455

In [6]:
def user_inventory(user_id,init_dic): #리스트형 user_id
    for i in notebook.tqdm(user_id):
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
        params = {
            'key' : '6C2D9CB7C9EEFF924BD9D7C22F1ED83B', 
            'steamid' : i,
            'include_appinfo' : '1',
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        try:
            user_inventory = r.json().get('response').get('games')
            if user_inventory == None:
                time.sleep(random.randint(2,4))
                continue
            init_dic.update({i:user_inventory})
            time.sleep(random.randint(2,4))
        except:
            continue

In [7]:
u_recent_app={}
user_inventory(user_id,u_recent_app)

In [8]:
final_user_inventory=del_app_detail(u_recent_app)
len(final_user_inventory)

1622

In [9]:
#df_u_recent_app= pd.DataFrame.from_dict(final_user_inventory, orient='index')
df= pd.DataFrame.from_dict(final_user_inventory, orient='index')
df=df.rename_axis('user_id').reset_index()

In [16]:
df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,4409,4410,4411,4412,4413,4414,4415,4416,4417,4418
0,76561198807142275,"{'appid': 227300, 'name': 'Euro Truck Simulato...","{'appid': 242920, 'name': 'Banished', 'playtim...","{'appid': 218620, 'name': 'PAYDAY 2', 'playtim...","{'appid': 271590, 'name': 'Grand Theft Auto V'...","{'appid': 730, 'name': 'Counter-Strike: Global...","{'appid': 292030, 'name': 'The Witcher 3: Wild...","{'appid': 550900, 'name': 'Metin2', 'playtime_...","{'appid': 578080, 'name': 'PLAYERUNKNOWN'S BAT...","{'appid': 648800, 'name': 'Raft', 'playtime_2w...",...,None,None,None,None,None,None,None,None,None,None
1,76561198099451925,"{'appid': 4000, 'name': 'Garry's Mod', 'playti...","{'appid': 4560, 'name': 'Company of Heroes - L...","{'appid': 228200, 'name': 'Company of Heroes '...","{'appid': 220, 'name': 'Half-Life 2', 'playtim...","{'appid': 340, 'name': 'Half-Life 2: Lost Coas...","{'appid': 320, 'name': 'Half-Life 2: Deathmatc...","{'appid': 360, 'name': 'Half-Life Deathmatch: ...","{'appid': 380, 'name': 'Half-Life 2: Episode O...","{'appid': 11450, 'name': 'Overlord', 'playtime...",...,None,None,None,None,None,None,None,None,None,None
2,76561198208050003,"{'appid': 4560, 'name': 'Company of Heroes - L...","{'appid': 9340, 'name': 'Company of Heroes: Op...","{'appid': 20540, 'name': 'Company of Heroes: T...","{'appid': 228200, 'name': 'Company of Heroes '...","{'appid': 8930, 'name': 'Sid Meier's Civilizat...","{'appid': 35130, 'name': 'Lara Croft and the G...","{'appid': 227300, 'name': 'Euro Truck Simulato...","{'appid': 47410, 'name': 'Stronghold Kingdoms'...","{'appid': 218230, 'name': 'PlanetSide 2', 'pla...",...,None,None,None,None,None,None,None,None,None,None
3,76561198033424233,"{'appid': 6860, 'name': 'Hitman: Blood Money',...","{'appid': 4560, 'name': 'Company of Heroes - L...","{'appid': 228200, 'name': 'Company of Heroes '...","{'appid': 320, 'name': 'Half-Life 2: Deathmatc...","{'appid': 340, 'name': 'Half-Life 2: Lost Coas...","{'appid': 20900, 'name': 'The Witcher: Enhance...","{'appid': 2820, 'name': 'X3: Terran Conflict',...","{'appid': 17470, 'name': 'Dead Space', 'playti...","{'appid': 24740, 'name': 'Burnout Paradise: Th...",...,None,None,None,None,None,None,None,None,None,None
4,76561198003838429,"{'appid': 240, 'name': 'Counter-Strike: Source...","{'appid': 4000, 'name': 'Garry's Mod', 'playti...","{'appid': 7940, 'name': 'Call of Duty 4: Moder...","{'appid': 8190, 'name': 'Just Cause 2', 'playt...","{'appid': 50130, 'name': 'Mafia II (Classic)',...","{'appid': 1030830, 'name': 'Mafia II: Definiti...","{'appid': 33910, 'name': 'Arma 2', 'playtime_f...","{'appid': 33930, 'name': 'Arma 2: Operation Ar...","{'appid': 219540, 'name': 'Arma 2: Operation A...",...,None,None,None,None,None,None,None,None,None,None


In [23]:
name_lst=[]
column_attr(df, name_lst)

In [24]:
len(name_lst)

13264

In [34]:
new_df=pd.DataFrame()

In [11]:
new_df=pd.DataFrame(df.index)
user_own_game(df,new_df,name_lst)

NameError: name 'name_lst' is not defined

In [ ]:
new_df.to_csv("./userinventory_fin.csv")

In [13]:
df.to_csv("./1126_Userinventory_Fin.csv")